# Homework 4
<p> Thomas Dougherty <br>
10-14-2023 </p>

## Problem 1

`What is transfer learning? Look it up, and write how you understood it.`
<p> Transfer learning is the act of training a model for one task (taking a pretrained model) and fine tuning it for another specific task. Transfer learning is a more efficient way of training neural networks because it allows them to use knowledge from the previous task while working with less labeled data. </br>

It's like going from a riding a mountain bike to learning how to ride a motorized dirt bike. All the time spent on a mountain bike taught you balance, cornering technique, and riding over rough terrain. Since those previous skills don't have to br learned again, you can focus on things like throttle control and riding at high speed giving you a headstart on the learning process. </p> </br>

`Explain where the differences are that make the loss plot different between the first two notebooks.`
1. neaclass2 is using a pretrained resnet18 model whereas neaclass1 is using a newly instantiated resnet18 model. This gives it a head start in the training process.
</br>
2. neaclass2 has learning rate scheduler. By slowing incrementally slowing down the learning rate over a period of epochs, the model will find the simple patterns first then "zero in" on finding complex patterns.
</br>

`Finally, run neaclass3.ipynb  and explain the differences with respect to neoclass2.ipynb. At the end of neaclass3.ipynb create a test dataset from  NEUdata_split/Test and  use it to evaluate the accuracy of both its models.`
1. neaclass3 deepcopies the original pre-trained resnet18 model.
2. neaclass3 
<br>
Accuracy for original Resnet18 model came out to 95%, whereas accuracy for the vector model came out to around 25%

In [1]:
import os
from utils import DatasetUtils
import torch
from torch import optim 
import torch.optim.lr_scheduler as lr_scheduler
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from PIL import Image
from torchvision import transforms
from torchvision.datasets import DatasetFolder
import cv2
import torch.nn as nn
import numpy as np


## Problem 2
### Design a convolutional autoencoder for a dataset of images 3 x 224 x224. Train it on NEU data for 50 epochs.

I'll start with getting the testing & training data and transforms

In [2]:
def load_image(img_path:str):    
        np_img = cv2.imread(img_path)
        return Image.fromarray(np_img)
"""
test_loader = DatasetUtils.create_loader_and_transform(
    root_path='RowanDLclassNEA/NEUdata_split/Test', 
    loader_func=load_image, 
    extensions=('.bmp',), 
    batch_size=32,
    shuffle=False,
    is_test=True
)

train_loader, val_loader, train_set, val_set = DatasetUtils.create_loader_and_transform(
    root_path='NEUdata', 
    loader_func=load_image, 
    extensions=('.bmp',), 
    batch_size=16,
    shuffle=True,
    is_test=False
)

"""
dset = DatasetFolder(root='NEUdata', loader = load_image, extensions = ('.bmp',))

transform = transforms.Compose([
  transforms.Resize(256),
  transforms.CenterCrop(224),
  transforms.ToTensor(),
  transforms.Normalize(
      mean=[0.485, 0.456, 0.406],
      std=[0.229, 0.224, 0.225])])


test_xform = transforms.Compose([
  transforms.ToTensor(),
  transforms.Normalize(
      mean=[0.485, 0.456, 0.406],
      std=[0.229, 0.224, 0.225])])

dset = DatasetFolder(root='NEUdata', loader = load_image, extensions = ('.bmp',), transform = transform)
test_data = DatasetFolder(root='RowanDLclassNEA/NEUdata_split/Test/', loader = load_image, extensions=('.bmp'), transform=test_xform)


train_set, val_set = random_split(
                      dset, 
                      [1200, 600])

train_loader = torch.utils.data.DataLoader(
                    train_set, 
                    batch_size=32, 
                    shuffle=True)

val_loader = torch.utils.data.DataLoader(
                    val_set, 
                    batch_size=32, 
                    shuffle=True)


test_loader = torch.utils.data.DataLoader(
            test_data,
            batch_size = 32,
            shuffle = False)

print(len(train_loader))
print(len(val_loader))
#print(len(test_loader))
print(train_set)


NameError: name 'test_data' is not defined

### Autoencorder Class

In [ ]:
import torch
import torch.nn as nn

class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        
        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, stride=2),
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, stride=2)
    )
        # Decoder
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(64, 3, kernel_size=2, stride=2),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x



### Training Loop

In [ ]:
# Initialize the model, loss, and optimizer
ae_model = Autoencoder()
device = device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
ae_model = ae_model.to(device)

#criterion = nn.MSELoss()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(ae_model.parameters(), 
                      lr=0.001, 
                      momentum=0.9)

# Training loop for 50 epochs
N_EPOCHS = 50
tr_loss_hist = []
val_loss_hist = []
for epoch in range(N_EPOCHS):

    # Training 
    train_loss = 0.0
    ae_model.train() # <1>
    for batch in train_loader:
        images,_ = batch
        images = images.cuda()
        optimizer.zero_grad()

        outputs = ae_model(images)
        loss = criterion(outputs, images)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    # Validation
    val_loss = 0.0
    ae_model.eval() # <2>
    for batch in val_loader:
        images,_ = batch
        images = images.cuda()

        outputs = ae_model(images)
        loss = criterion(outputs, images)

        val_loss += loss.item()

    print("Epoch: {} Train Loss: {} Val Loss: {}".format(
                  epoch, 
                  train_loss/len(train_loader), 
                  val_loss/len(val_loader)))
    tr_loss_hist.append(train_loss/len(train_loader))
    val_loss_hist.append(val_loss/len(val_loader))

### Testing

In [ ]:
num_correct = 0.0

for x_test_batch, y_test_batch in test_loader:

    ae_model.eval()
    y_test_batch = y_test_batch.to(device)
    x_test_batch = x_test_batch.to(device)

    y_pred_batch = ae_model(x_test_batch)
    _, predicted = torch.max(y_pred_batch, 1)
    num_correct += (predicted == y_test_batch).float().sum()

lenet_accuracy = num_correct/(len(test_loader)*test_loader.batch_size)

print(len(test_loader), test_loader.batch_size)